# TEST

In [1]:
from barbar import Bar
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import utils

## 1. Split train / validation

In [2]:
split = 0.8
all_videos = np.array(utils.get_train_test_video_names()[0])

In [3]:
train_videos = np.array(all_videos)[np.random.choice(len(all_videos), int(0.8 * len(all_videos)))]
validation_videos = np.setdiff1d(all_videos, train_videos, assume_unique=False)
train_videos.sort()
validation_videos.sort()

In [4]:
train_data = utils.HernitiaDataset(videonames = train_videos, resize = (60,80), mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
validation_data = utils.HernitiaDataset(videonames = validation_videos, resize = (60,80), mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])

## 2. Model construction

In [5]:
class MyFC(torch.nn.Module):
    def __init__(self, num_classes):
        super(MyFC, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(256,num_classes),
        )

    def forward(self, x):
        return self.classifier(x)

resnet18 = models.resnet18(pretrained=True)
resnet18.fc = MyFC(utils.num_classes)

for name, param in resnet18.named_parameters():
    if 'fc' not in name:
        param.requires_grad=False

In [6]:
# training parameters
LEARNING_RATE = 0.01
EPOCHS = 1

# instantiate data loaders
train_loader = utils.DataLoader(dataset=train_data, batch_size=1, shuffle=False)
validation_loader = utils.DataLoader(dataset=validation_data, batch_size=1, shuffle=False)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(resnet18.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = "min", patience = 1, verbose = True)

## 3. Useful functions

In [7]:
def binary_acc(y_pred, y_validation):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_validation).sum().float()
    acc = correct_results_sum/y_validation.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [8]:
# Training
def train(model, epoch):
    print('\nEpoch: %d' % epoch)
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(Bar(train_loader)):
        inputs, targets = inputs.to(utils.device), targets.to(utils.device)
        optimizer.zero_grad()
        outputs = model(inputs)

        loss = criterion(outputs, targets.unsqueeze(1))
        acc = binary_acc(outputs, targets.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    print(f'Epoch {epoch+0:03}: | Training loss: {epoch_loss/len(train_loader):.5f} | Training acc: {epoch_acc/len(train_loader):.3f}')
    return epoch_loss/len(train_loader), epoch_acc/len(train_loader)

In [9]:
def validate(model, epoch, model_name = 'ResNet18'):
    global best_acc
    model.eval()
    validation_loss = 0
    validation_acc = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(Bar(validation_loader)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets.unsqueeze(1))
            acc = binary_acc(outputs, targets.unsqueeze(1))

            validation_loss += loss.item()
            validation_acc += acc.item()

        print(f'Epoch {epoch+0:03}: | validation loss: {validation_loss/len(validation_loader):.5f} | validation acc: {validation_acc/len(validation_loader):.3f}')

    # Save checkpoint.
    acc = validation_acc/len(validation_loader)
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/' + model_name + '.pth')
        best_acc = acc
    
    return validation_loss/len(validation_loader), validation_acc/len(validation_loader)

In [10]:

def plot_performance(model_name, training_loss, training_acc, validation_loss, validation_acc):

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18,6))

    # subplot for losses
    ax1.grid()
    ax1.set_xlabel('epoch')
    ax1.set_ylabel('loss')
    ax1.plot(training_loss, color = 'blue', label = "training loss")
    ax1.plot(validation_loss, color = 'green', label = "validation loss")
    ax1.legend()

    # subplot for losses
    ax2.grid()
    ax2.set_xlabel('epoch')
    ax2.set_ylabel('accuracy')
    ax2.plot(training_acc, color = 'blue', label = "training accuracy")
    ax2.plot(validation_acc, color = 'green', label = "validation accuracy")
    ax2.legend()

    plt.subplots_adjust(wspace=0.5)
    plt.suptitle(model_name)
    plt.savefig('figures/' + model_name + '.png')
    plt.show()

## 3.bis Test code

In [11]:
for batch_idx, (inputs, targets) in enumerate(train_data):
    print("test")

100%|██████████| 1540/1540 [00:08<00:00, 183.03it/s]


test


100%|██████████| 599/599 [00:03<00:00, 191.46it/s]


test


100%|██████████| 800/800 [00:04<00:00, 193.36it/s]


test


100%|██████████| 800/800 [00:03<00:00, 205.93it/s]


test


 98%|█████████▊| 4480/4568 [00:23<00:00, 190.91it/s]


KeyboardInterrupt: 

## 4. Train

In [24]:
model = resnet18

In [25]:
train_loss, validation_loss = [], []
train_acc, validation_acc = [], []
best_acc = 0

for epoch in range(EPOCHS):
    trnl, trnacc = train(model, epoch)
    tstl, tstacc = validate(model, epoch, 'ResNet18')
    train_loss.append(trnl)
    train_acc.append(trnacc)
    validation_loss.append(tstl)
    validation_acc.append(tstacc)
    scheduler.step(tstl)



Epoch: 0


AttributeError: 